# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
# !pip install -U pandas

In [14]:
# import libraries
import pandas as pd
import re
import nltk
import numpy as np

import sqlite3
from sqlalchemy import create_engine

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score, log_loss
from sklearn.preprocessing import MultiLabelBinarizer

from scipy import sparse

# from imblearn.combine import SMOTETomek
# from imblearn.over_sampling import SMOTE

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
# load data from database
# engine = create_engine('sqlite:///InsertDatabaseName.db')
conn = sqlite3.connect('InsertDatabaseName.db')
query = 'SELECT * FROM InsertTableName'
df = pd.read_sql(query, con=conn)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    # find urls and replace them with 'urlplaceholder'
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex, 'urlplaceholder', text)
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
# split into train an test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [19]:
model = pipeline
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
y_pred = model.predict(X_test)

In [21]:
# transform iterable of iterables to a multilabel format
m = MultiLabelBinarizer().fit(np.array(y_test))

In [22]:
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

In [24]:
# print classification report for each feature   
print(classification_report(y_test, y_pred, target_names=y_test.keys()))

# print accuracy, Note: Accuracy does not work well on imbalanced data sets!
print(f"Accuracy of Model: {model.score(X_test,np.array(y_test))} - bad metrics (!)")

# print f1 score
print("F1 Score: ", f1_score(m.transform(np.array(y_test)), m.transform(y_pred), average='macro'))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      5020
               request       0.82      0.44      0.57      1135
                 offer       0.00      0.00      0.00        27
           aid_related       0.76      0.59      0.67      2711
          medical_help       0.63      0.10      0.17       533
      medical_products       0.67      0.09      0.16       305
     search_and_rescue       0.57      0.08      0.13       172
              security       0.60      0.03      0.05       120
              military       0.51      0.12      0.19       201
                 water       0.81      0.20      0.32       415
                  food       0.86      0.42      0.56       729
               shelter       0.79      0.38      0.51       544
              clothing       0.62      0.10      0.17        99
                 money       0.50      0.01      0.03       142
        missing_people       0.75      

In [25]:
# print log los
# log loss nearer 0 indicates higher accuracy
log_loss(m.transform(np.array(y_test)), m.transform(y_pred))

3.017831260000666

### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [29]:
parameters = {
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__min_samples_leaf': [5, 10, 20],
    'clf__estimator__max_features': [0.5, 1, "log2"]
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=-1)
cv.fit(X_train, y_train)
y_pred_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [30]:
# print classification report for each feature   
print(classification_report(y_test, y_pred, target_names=y_test.keys()))

# print accuracy, Note: Accuracy does not work well on imbalanced data sets!
print(f"Accuracy of Model: {model.score(X_test,np.array(y_test))} - bad metrics (!)")

# print f1 score
print("F1 Score: ", f1_score(m.transform(np.array(y_test)), m.transform(y_pred), average='macro'))


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      5020
               request       0.82      0.44      0.57      1135
                 offer       0.00      0.00      0.00        27
           aid_related       0.76      0.59      0.67      2711
          medical_help       0.63      0.10      0.17       533
      medical_products       0.67      0.09      0.16       305
     search_and_rescue       0.57      0.08      0.13       172
              security       0.60      0.03      0.05       120
              military       0.51      0.12      0.19       201
                 water       0.81      0.20      0.32       415
                  food       0.86      0.42      0.56       729
               shelter       0.79      0.38      0.51       544
              clothing       0.62      0.10      0.17        99
                 money       0.50      0.01      0.03       142
        missing_people       0.75      

In [31]:
# print log loss
# log loss nearer 0 indicates higher accuracy
log_loss(m.transform(np.array(y_test)), m.transform(y_pred_cv))

2.8817662826368218

In [ ]:
cv.best_params_

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# build pipeline
pipe_balanced = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
])

In [ ]:
# fit new model
model_b = pipe_balanced
model_b.fit(X_train, y_train)

In [ ]:
# test new model
y_pred_b = model_b.predict(X_test)

In [ ]:
# print classification report for each row
i = 0
for index, row in y_test.iterrows():
    print(classification_report(row, y_pred_b[i], labels=np.unique(y_pred_b)))
    i += 1

# print accuracy, Note: Accuracy does not work well on imbalanced data sets!!!
print(f"Accuracy of Model: {model_b.score(X_test,np.array(y_test))} - bad metrics (!)")

# print f1 score
print("F1 Score: ", f1_score(m.transform(np.array(y_test)), m.transform(y_pred_b), average='macro'))
print(i)

In [ ]:
# print log los
# log loss nearer 0 indicates higher accuracy
log_loss(m.transform(np.array(y_test)), m.transform(y_pred_b))

#### 8.1 MLSMOTE

In [ ]:
X_train.head()

In [ ]:
import MLSMOTE

In [ ]:
# initialize vectorizer and tfidt transformer
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()

# vectorize and generate tfidf of trainings data
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf_matrix = tfidf.fit_transform(X_train_counts)

# convert matrix into dataframe
X_train_tfidf = pd.DataFrame.sparse.from_spmatrix(X_train_tfidf_matrix)

In [ ]:
# Getting minority samples of the dataframe
X_sub, y_sub = MLSMOTE.get_minority_instace(X_train_tfidf, y_train)

## Generating synthetic samples based on the minority samples
#X_res, y_res = mlsmote.MLSMOTE(X_sub, X_sub, 800, 5)

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.